In [1]:
5+5

10

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\user\anaconda3\envs\medical_chatbot_opensource_llm_llama2\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [23]:
# embedding model in dimension is 384, this will be useful while creating PINECONE_API_ENV

PINECONE_API_KEY = 'XXXXXXXXXXXXXX'
PINECONE_API_ENV = 'XXXXXX' 

embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"


In [4]:
#extract the data from pdf

def load_pdf(data):
   loader =  DirectoryLoader(data,
                    glob = '*.pdf',
                    loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents


In [5]:
extracted_data = load_pdf('data/')

In [6]:
type(extracted_data)

list

In [7]:
#page wise data

extracted_data[2].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1'

In [8]:
# create text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks




In [9]:
text_chunks = text_split(extracted_data)
print('length of text chunks :', len(text_chunks))

length of text chunks : 7020


In [10]:
type(text_chunks)

list

In [11]:
# download embedding model

def download_hugging_face_embeddings(embedding_model_name):
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    return embeddings



In [12]:
embeddings = download_hugging_face_embeddings(embedding_model_name)

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result = embeddings.embed_query('Hello world')
print('Length : ', len(query_result))

Length :  384


In [15]:
query_result


[-0.03447727859020233,
 0.03102315217256546,
 0.006734953261911869,
 0.02610895223915577,
 -0.03936200961470604,
 -0.16030248999595642,
 0.06692399084568024,
 -0.006441458594053984,
 -0.04745051637291908,
 0.014758842997252941,
 0.07087532430887222,
 0.055527545511722565,
 0.019193314015865326,
 -0.02625134028494358,
 -0.010109533555805683,
 -0.026940464973449707,
 0.022307470440864563,
 -0.022226575762033463,
 -0.1496926248073578,
 -0.0174929928034544,
 0.007676254957914352,
 0.0543522909283638,
 0.003254421753808856,
 0.03172592446208,
 -0.08462145179510117,
 -0.0294059906154871,
 0.051595594733953476,
 0.04812406003475189,
 -0.003314807778224349,
 -0.05827919393777847,
 0.04196925461292267,
 0.02221064642071724,
 0.1281888782978058,
 -0.02233896031975746,
 -0.011656253598630428,
 0.06292839348316193,
 -0.032876305282115936,
 -0.0912260115146637,
 -0.031175408512353897,
 0.05269954353570938,
 0.04703480377793312,
 -0.08420310914516449,
 -0.030056172981858253,
 -0.020744821056723595,


In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot-opensource-llm-llama2"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [24]:
# if we already have an index, we can load it like this.
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot-opensource-llm-llama2"

docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "what are allergies ?"

docs = docsearch.similarity_search(query, k=3)

print("Result : ", docs)

Result :  [Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-', metadata={}), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\

In [25]:
# send these to LLM ang get the correct answer

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [26]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context" , "question"])
chain_type_kwargs={"prompt": PROMPT}

In [27]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama",
                    config = {'max_new_tokens':512,
                          'temperature':0.8})


In [28]:
# create question answering object

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [30]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Allergy is an abnormal response of the immune system to a substance (allergen) that is typically harmless to most people. When this occurs, the immune system produces antibodies to fight off the allergen, leading to a range of symptoms including sneezing, congestion, runny nose, itchy eyes, and difficulty breathing.


In [ ]:
# ask a question

# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])